# <p style="text-align: center;"> AP EKF roll and pitch angles and bias estimation

In [30]:
import sympy as sp
import numpy as np
from scipy import linalg
from sympy import *
from IPython.display import Math, Latex

## List of tuning parameters for roll and pitch fusion
#### A. Process noise parameters
1. $GYRO\_P\_NSE$ : Rate gyro noise [rad/s] $(AP \; value: 0.03)$ <br>
&emsp; <b>AP definition:</b> This control disturbance noise controls the growth of estimated error due to gyro measurement errors excluding bias. Increasing it makes the flter trust the gyro measurements less and other measurements more. <br>
<font color = red> <i><b>Interpretation:</b> This parameter dictates the amount of deviation from the motion model (of angular velocities) that the estimated model (of angular velocities) can have, i.e., whether the estimated model (of angular velocities) will follow the motion model (of angular velocities) or the true values (of angular velocities).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of angular velocities) will follow the true model (of angular velocities) and will be more sensitive to noisy measurements.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of angular velocities) will follow the motion model (of angular velocities), will be less sensitive to noisy measurements, and rapid changes in the true model (of angular velocities) will be ignored (which should actually be tracked.).</i> </font> <br>
</br>
2. $GBIAS\_P\_NSE$ : Rate gyro bias stability [rad/s/s] $(AP \; value: 0.0005)$ <br>
&emsp; <b>AP definition:</b> This state  process noise controls growth of the gyro delta angle bias state error estimate. Increasing it makes rate gyro bias estimation faster and noisier. <br>
<font color = red> <i><b>Interpretation:</b> This parameter dictates the amount of deviation from the motion model (of angular velocities bias) that the estimated model (of angular velocities bias) can have, i.e., whether the estimated model (of angular velocities bias) will follow the motion model (of angular velocities bias) or the true values (of angular velocities bias).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of angular velocities bias) will follow the true model (of angular velocities bias) and will be more sensitive to noisy measurements.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of angular velocities bias) will follow the motion model (of angular velocities bias), will be less sensitive to noisy measurements, and rapid changes in the true model (of angular velocities bias) will be ignored (which should actually be tracked.).</i> </font>

#### B. Measurement uncertainty parameters
3. $angErr$ : 1-sigma spherical angle error [rad] $(AP \; value: -)$ <br>
<font color = red> <i><b>Interpretation:</b> This parameter denotes the angular velocity measurement accuracy of the gyroscope. It dictates whether the estimated model (of angular velocities) will follow the motion model (of angular velocities) or the true measurements (of angular velocities).</i> </font> <br>
<font color = red> <i><b>Increasing the parameter:</b> The estimated model (of angular velocities) will favor the motion model (of angular velocities) and rapid changes in the true measurements (of angular velocities) will be ignored.</i> </font> <br>
<font color = red> <i><b>Decreasing the parameter:</b> The estimated model (of angular velocities) will favor the true measurements (of angular velocities).</i> </font>

## System model

The system model is represented in the form of the state extrapolation equation: <br>
<p style="text-align: center;"> $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$ <br> </p>

where, <br>
$X_{t|t-1}$ = Predicted system state at time t <br>
$F$ = State transition matrix <br>
$X_{t-1|t-1}$ = System state at time t-1 <br>
$G$ = Control matrix <br>
$U_{t-1|t-1}$ = Control vector at time t-1

In [3]:
# XT_predict_compact = Predicted system state at time t
# F = State transition matrix
# XT_previous = System state at time t-1
# G = Control matrix
# U = Control vector at time t-1

roll_t1, roll_bias_t1, pitch_t1, pitch_bias_t1, roll_t, roll_rate_t, roll_bias_t, pitch_t, pitch_rate_t, pitch_bias_t, delt = sp.symbols('\phi_{t|t-1}, \phi_{b|t|t-1}, \\theta_{t|t-1}, \\theta_{b|t|t-1}, \phi_{t-1}, \dot{\phi}_{t-1}, \phi_{b|t-1}, \\theta_{t-1}, \dot{\\theta}_{t-1}, \\theta_{b|t-1}, \Delta{t}')

Xt_predict_compact = sp.Matrix([[roll_t1],[roll_bias_t1],[pitch_t1],[pitch_bias_t1]])
F = sp.Matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
Xt_previous = sp.Matrix([[roll_t],[roll_bias_t],[pitch_t],[pitch_bias_t]])
G = sp.Matrix([[delt,0],[0,0],[0,delt],[0,0]])
Ut_previous = sp.Matrix([[roll_rate_t],[pitch_rate_t]])

display(Math(r'X_{{t|t-1}} = {}, F = {}, X_{{t-1|t-1}} = {}, G = {}, U_{{t-1|t-1}} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

Arranging the matrices in the form of the state extrapolation equation: $X_{t|t-1} = F.X_{t-1|t-1} + G.U_{t-1|t-1}$

In [4]:
display(Math(r'{} = {}{} + {}{}'.format(sp.latex(Xt_predict_compact), sp.latex(F), sp.latex(Xt_previous), sp.latex(G), sp.latex(Ut_previous))))

<IPython.core.display.Math object>

## Predicted system state $X$ (at time t-1, for time t)

In [5]:
Xt_predict = F*Xt_previous+G*Ut_previous

display(Math(r'X_{{t|t-1}} = {} = {}'.format(sp.latex(Xt_predict_compact), sp.latex(Xt_predict))))

<IPython.core.display.Math object>

## Process noise

In [6]:
# Q = Process noise covariance matrix
# q = Process noise standard deviation (sigma) of each state variable

Q_roll, Q_roll_bias, Q_pitch, Q_pitch_bias = sp.symbols('q_{\phi}^2, q_{{\phi}|b}^2, q_{\\theta}^2, q_{{\\theta}|b}^2')

Q = sp.Matrix([[Q_roll,0,0,0],[0,Q_roll_bias,0,0],[0,0,Q_pitch,0],[0,0,0,Q_pitch_bias]])

display(Math(r'Q = {}'.format(sp.latex(Q))))

<IPython.core.display.Math object>

The process noise covariance matrix and its variables in the AP code are as follows:

In [7]:
Q_roll_AP, Q_roll_bias_AP, Q_pitch_AP, Q_pitch_bias_AP = sp.symbols('{(gyrNoise*\Delta{t})}^2, {(gyroBiasProcessNoise*(\Delta{t})^2)}^2, {(gyrNoise*\Delta{t})}^2, {(gyroBiasProcessNoise*(\Delta{t})^2)}^2')

Q_AP = sp.Matrix([[Q_roll_AP,0,0,0],[0,Q_roll_bias_AP,0,0],[0,0,Q_pitch_AP,0],[0,0,0,Q_pitch_bias_AP]])

display(Math(r'Q_{{AP}} = {}'.format(sp.latex(Q_AP))))

<IPython.core.display.Math object>

$gyrNoise = GYRO\_P\_NSE$ <br>
$gyroBiasProcessNoise = GBIAS\_P\_NSE$ <br>

## Measurement uncertainty

In [8]:
# R = Measurement uncertainty covariance matrix
# r = Measurement uncertainty standard deviation (sigma) of each sensor that can measure a system state variable

R_roll, R_roll_rate, R_pitch, R_pitch_rate = sp.symbols('r_{\phi}^2, r_{\dot{\phi}}^2, r_{\\theta}^2, r_{\dot{\\theta}}^2')

R = sp.Matrix([[R_roll,0,0,0],[0,R_roll_rate,0,0],[0,0,R_pitch,0],[0,0,0,R_pitch_rate]])

display(Math(r'R = {}'.format(sp.latex(R))))

<IPython.core.display.Math object>

The measurement uncertainty covariance matrix and its variables in the AP code are as follows:

In [29]:
R_angle_AP, R_angle_rate_AP = sp.symbols('{(angErr)}^2, {(angErr*\Delta{t})}^2')

R_AP = sp.Matrix([[R_angle_AP,0,0,0],[0,R_angle_rate_AP,0,0],[0,0,R_angle_AP,0],[0,0,0,R_angle_rate_AP]])

display(Math(r'R_{{AP}} = {}'.format(sp.latex(R_AP))))

<IPython.core.display.Math object>

## Initialized estimation uncertainty covariance matrix $P$ (at time t-1, for time t-1)

In [10]:
# Pt_previous = Estimated estimation uncertainty covariance matrix of the system state variables at time t-1

P_roll, P_roll_bias, P_pitch, P_pitch_bias = sp.symbols('P_{{\phi}|t-1|t-1}, P_{{\phi}|b|t-1|t-1}, P_{{\\theta}|t-1|t-1}, P_{{\\theta}|b|t-1|t-1}')

Pt_previous = sp.Matrix([[P_roll,0,0,0],[0,P_roll_bias,0,0],[0,0,P_pitch,0],[0,0,0,P_pitch_bias]])

display(Math(r'P_{{t-1|t-1}} = {}'.format(sp.latex(Pt_previous))))

<IPython.core.display.Math object>

## Predicted estimation uncertainty covariance matrix $P$ (at time t-1, for time t)

In [11]:
# Pt_predict = Predicted estimation uncertainty covariance matrix of the system state variables at time t

Pt_predict = (F*Pt_previous*F.T + Q)

display(Math(r'P_{{t|t-1}} = {}'.format(sp.latex(Pt_predict))))

<IPython.core.display.Math object>

## Updated measurement vector (at time t, for time t)

In [14]:
# Zt_update = Measurement vector at time t containing the sensor measurements
# H = Observation matrix to map the measurement vector to the system state variables

roll_t, roll_rate_t, pitch_t, pitch_rate_t = sp.symbols('z_{{\phi}|t}, z_{\dot{\phi}|t}, z_{{\\theta}|t}, z_{\dot{\\theta}|t}')

Zt_update = sp.Matrix([[roll_t],[roll_rate_t],[pitch_t],[pitch_rate_t]])
H = sp.Matrix([[1,0,0,0],[0,0,0,0],[0,0,1,0],[0,0,0,0]])

display(Math(r'Z_{{t|t}} = {}, H = {}'.format(sp.latex(Zt_update), sp.latex(H))))

<IPython.core.display.Math object>

The measurement vector and its variables in the AP code are as follows:

In [19]:
roll_t_AP, roll_rate_t_AP, pitch_t_AP, pitch_rate_t_AP = sp.symbols('imuDataDelayed.delAng.x, imuDataDelayed.delAng.x/imuDataDelayed.delAngDT, imuDataDelayed.delAng.y, imuDataDelayed.delAng.y/imuDataDelayed.delAngDT')

Zt_update_AP = sp.Matrix([[roll_t_AP],[roll_rate_t_AP],[pitch_t_AP],[pitch_rate_t_AP]])
display(Math(r'Z_{{t|t|AP}} = {}'.format(sp.latex(Zt_update_AP))))

<IPython.core.display.Math object>

## Updated innovation matrix (at time t, for time t)

In [20]:
# Yt_update = Updated innovation matrix at time t

Yt_update = H*Xt_predict_compact - Zt_update

display(Math(r'Y_{{t|t}} = {}'.format(sp.latex(Yt_update))))

<IPython.core.display.Math object>

## Updated innovation covariance matrix (at time t, for time t)

In [21]:
# St_update = Updated innovation covariance matrix at time t

St_update = H*Pt_predict*H.T + R

display(Math(r'S_{{t|t}} = {}'.format(sp.latex(St_update))))

<IPython.core.display.Math object>

In [22]:
Num = sp.adjoint(St_update).simplify()

In [23]:
Den = sp.det(St_update).simplify()

In [24]:
St_inv = (Num/Den).simplify()

display(Math(r'S_{{t|t}}^{{-1}} = {}'.format(sp.latex(St_inv))))

<IPython.core.display.Math object>

## Updated Kalman gain $K$ (at time t, for time t)

In [25]:
# Kt_update = Updated Kalman gain at time t

Kt_update = (Pt_predict*H.T*St_inv).simplify()

display(Math(r'K_{{t|t}} = {}'.format(sp.latex(Kt_update))))

<IPython.core.display.Math object>

## Estimated system state $X$ (at time t, for time t)

In [26]:
# Xt_estimate = Estimated system state at time t

Xt_estimate = (Xt_predict_compact + Kt_update*(Yt_update))

display(Math(r'X_{{t|t}} = {}'.format(sp.latex(Xt_estimate))))

<IPython.core.display.Math object>

## Estimated estimation uncertainty covariance matrix $P$ (at time t, for time t)

In [28]:
# Pt_estimate = Estimated estimation uncertainty matrix of the system state variables at time t

I = sp.Matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])

Pt_estimate = ((I - Kt_update*H)*Pt_predict)

display(Math(r'P_{{t|t}} = {}'.format(sp.latex(Pt_estimate))))

<IPython.core.display.Math object>